In [ ]:
from numba import jit, vectorize, float32, float64,guvectorize
import numpy as np
from scipy import spatial, integrate, optimize
from matplotlib import pyplot as plt
from SmoothingLength import HsmlIter

normdict = {1: 4./3, 3: 8./np.pi, 2: 40/(7*np.pi)}

@vectorize([float32(float32), float64(float64)])
def Kernel(q):
    if q <= 0.5:
        return 1 - 6*q**2 + 6*q**3
    elif q <= 1.0:
        return 2 * (1-q)**3
    else: return 0.0

@vectorize([float32(float32), float64(float64)])
def DKernel(q):
    if q <= 0.5:
        return -12*q + 18*q*q
    elif q <= 1.0:
        return -6 * (1-q)**2
    else: return 0.0

@vectorize([float32(float32), float64(float64)])
def Kernel3D(q):
    if q <= 0.5:
        return 2.5464790894703255 * (1 - 6*q**2 + 6*q**3)
    elif q <= 1.0:
        return 2.5464790894703255 * 2 * (1-q)**3
    else: return 0.0
    
@vectorize([float32(float32), float64(float64)])
def DKernel3D(q):
    if q <= 0.5:
        return 2.5464790894703255 * (-12*q + 18*q**2)
    elif q <= 1.0:
        return -2.5464790894703255 * 6 * (1-q)**2
    else: return 0.0

@jit
def HydroForce(x, m, f, K, dK, rho, h, ngb, ngbdist):
    A = 1.0
    N, dim = x.shape
    Nngb = ngbdist.shape[1]
    a = np.zeros_like(x)
    for i in xrange(N):
        ngbi = ngb[i]
        xngb = x[ngbi]
        mngb = m[ngbi]
        hngb = h[ngbi]
        rngb = ngbdist[i]
        fngb = f[ngbi]
        Ki = K[i]
        dKi = dK[i]
        hi = h[i]
        fi = f[i]
        rhoi = rho[i]
        for j in xrange(1,Nngb):
            atot = mngb[j] * (fi * A * rhoi**(gamma-2.0) * dKi[j] / hi**(dim+1)
            + fngb[j] * A * rho[ngbi[j]]**(gamma-2.0) * norm*DKernel(rngb[j]/h[ngbi[j]]) / h[ngbi[j]]**(dim+1))
            #atot = mngb[j] * (rhoi**(gamma-2.0) + rho[ngbi[j]]**(gamma-2.0)) * dKi[j] / hi**4.0
            for k in xrange(dim):
                dx1 = xngb[j,k] - xngb[0,k]
                dx2 = -np.sign(dx1) * (1.0 - np.abs(dx1))
                if np.abs(dx1) < np.abs(dx2):
                    dx = dx1
                else:
                    dx = dx2
                a[i,k] += atot*dx/rngb[j]
                
    return a


@jit
def KernelSum(f,r,h):
    return np.sum(f*norm*Kernel(r/h)/h**dim)

In [ ]:
N=  100
#x = np.random.rand(N,1)**0.5
#x = (np.arange(32)/32.)
#x = x + 0.001*np.sin(2*np.pi*x)
#y = np.arange(32)/32.
#x,y,z = np.meshgrid(x,y,y)
#x = np.c_[x.flatten(),y.flatten(),z.flatten()]
#x = (x)%1.0
dim = 1
norm = normdict[dim]
dtmin = (5./3)**0.5 / N /10

T = 1./(5./3)**0.5
nsteps = int(T/dtmin+1)
print nsteps
dt = T/nsteps
m = np.ones(N)/N
gamma = 5./3
x = np.array([np.arange(0,N)/float(N),]).T

x += 0.001*np.sin(2*np.pi*x)
x = x % 1.0
#tree = spatial.cKDTree(x,boxsize=1.0)
des_ngb = 8
v = np.zeros_like(x)
tree = spatial.cKDTree(x, boxsize=1.)
ngbdist, ngb = tree.query(x,des_ngb)
h = HsmlIter(ngb,ngbdist, error_norm=1e-12,dim=1)
rho0 = m * des_ngb / (2*h)

@jit
def HForce(x, m, h):
    N, dim = x.shape
    tree = spatial.cKDTree(x, boxsize=1.)
    ngbdist, ngb = tree.query(x, des_ngb)
    h = HsmlIter(ngb,ngbdist, error_norm=1e-12,dim=dim)
    for i in xrange(N):
        rho = m[i] * des_ngb / (2*h)
        fi = 0
        q = ngbdist[i]/h[i]
        K = norm*Kernel(q)
        dK = norm*Kernel(q)
        for j in xrange(des_ngb):
            fi += -m[ngb[i,j]] * h[i]**-(dim+1) * (dK * q )

In [ ]:
tree = spatial.cKDTree(x, boxsize=1.)
ngbdist, ngb = tree.query(x,des_ngb)
h = HsmlIter(ngb,ngbdist, error_norm=1e-12,dim=1)
rho = m * des_ngb / (2*h)
q = ngbdist/h[:,np.newaxis]
K = norm*Kernel(q)
dK = norm*DKernel(q)
FI = (1 + h / (3*rho) * np.sum((-m[:,np.newaxis]*h[:,np.newaxis]**-(dim+1) * (dK *q + dim*K)),axis=1))**-1
F = HydroForce(x,m,FI, K, dK, rho, h, ngb, ngbdist)

In [ ]:
e1 = []
e2 = []
#plt.scatter(x, -np.gradient(rho**gamma)/np.gradient(x[:,0]) / rho * 100.*dt)
for i in xrange(nsteps*2):
    x += v * dt + F * dt**2 / 2
    x[:,0] = x[:,0]%1.0%1.0
    v += F * dt/2
    tree = spatial.cKDTree(x, boxsize=1.)
    ngbdist, ngb = tree.query(x,des_ngb)
    h = HsmlIter(ngb,ngbdist, error_norm=1e-12,dim=1)
    rho = m * des_ngb / (2*h)
    q = ngbdist/h[:,np.newaxis]
    K = norm*Kernel(q)
    dK = norm*DKernel(q)
    FI = (1 + h / (3*rho) * np.sum((-m[:,np.newaxis]*h[:,np.newaxis]**-(dim+1) * (dK *q + dim*K)),axis=1))**-1
    F = HydroForce(x,m,FI, K, dK, rho, h, ngb, ngbdist)
    v += F * dt/2
    e1.append(np.sum(0.5*m*v**2))
    e2.append(np.sum(rho**(5./3) * 3./2 * m / 2 / h))
    #e.append(np.abs(np.sum(F,axis=0)))
    #e.append(np.sum(0.5*m*v**2) + np.sum(3./2 * rho**(5./3)))
#plt.scatter(np.arange(100), np.array(e) - e[0])
#plt.yscale('log')

plt.plot((np.array(e1) + np.array(e2))/np.array(e2))
plt.show()
print (rho-rho0).std()
#plt.yscale('log')
#plt.ylim(0.1, 10)
#plt.scatter(x[:,0],np.gradient(rho**gamma)/np.gradient(x[:,0]) / rho, color='blue')

#if i % (nsteps/10)==0:
    #if i%10==0:
    #    plt.scatter(x[:,0],v[:,0])
    #
#plt.scatter(x[:,0][(x[:,1]==0.0)*(x[:,2]==0.0)], )
        #plt.show()

In [ ]:
des_ngb = 32
x = np.arange(200)/200.
x, y = np.meshgrid(x, x)
x = np.c_[x.flatten(), y.flatten()]
tree = spatial.cKDTree(x, boxsize=1.0)
ngbdist, ngb = tree.query(x, des_ngb)
h = HsmlIter(ngb, ngbdist, error_norm=1e-12, dim=2)
#rhos.append(np.mean(des_ngb / 200.**2 / (np.pi*h**2)))
    
#pt.plot(range(10,120), rhos)
#plt.show()

In [ ]:
#@guvectorize([(float64[:], float64)], '(n),()')
@vectorize([float32(float32,float32), float64(float64,float64)])
def Weights(ngbdist, h):
    k = Kernel(ngbdist/h)
    return np.sum(k,axis=0)


In [17]:
from SmoothingLength import HsmlIter
from scipy import spatial
import numpy as np
from time import time
from numba import autojit, jit, vectorize, float32, float64

@jit
def DX(x, ngb, boxsize):
    N, dim, des_ngb = x.shape[0], x.shape[1], ngb.shape[1]
    dx = np.empty((N, des_ngb,dim))
    for i in xrange(N):
        ngbi = ngb[i]
        for j in xrange(des_ngb):
            xngb = x[ngbi[j]]
            for k in xrange(dim):
                dx[i,j,k] = xngb[k] - x[i,k]
                if np.abs(dx[i,j,k]) > boxsize/2:
                    dx[i,j,k] = -np.sign(dx[i,j,k])*(boxsize - np.abs(dx[i,j,k]))
    return dx

@vectorize([float32(float32), float64(float64)])
def Kernel(q):
    if q <= 0.5:
        return 1 - 6*q**2 + 6*q**3
    elif q <= 1.0:
        return 2 * (1-q)**3
    else: return 0.0

@vectorize([float32(float32), float64(float64)])
def DKernel(q):
    if q <= 0.5:
        return -12*q + 18*q*q
    elif q <= 1.0:
        return -6 * (1-q)**2
    else: return 0.0

@jit
def Periodicize(dx, boxsize):
    for i in xrange(dx.size):
        if np.abs(dx[i]) > boxsize/2:
            dx[i] = -np.sign(dx[i])*(boxsize - np.abs(dx[i]))

@jit
def DF(f, ngb):
    df = np.empty(ngb.shape)
    for i in xrange(ngb.shape[0]):
        for j in xrange(ngb.shape[1]):
            df[i,j] = f[ngb[i,j]] - f[i]
    return df

@jit
def SymmetricForce(P, rho, ngb, dweights):
    dP = DF(P, ngb)
    N, des_ngb, dim = dweights.shape
    F = np.zeros((N, dim))
    for i in xrange(N):
        for j in xrange(des_ngb):
            n=ngb[i,j]
            for k in xrange(dim):
                F[i,k] -= 0.5*dweights[i,j,k]*dP[i,j]/rho[i]
                F[n,k] -= 0.5*dweights[i,j,k]*dP[i,j]/rho[i]
    return F
                
class ParticleSystem(object):
    def __init__(self, m, x, v, u=None, A=None, des_ngb=None, gamma=5./3,boxsize=1.0): 
        self.N, self.dim = x.shape
        if des_ngb is None:
            des_ngb = {1: 8, 2: 20, 3:32}[self.dim]
 
        self.volnorm = {1: 2.0, 2: np.pi, 3: 4*np.pi}[self.dim]
        
        self.gamma = gamma
        
        self.des_ngb = des_ngb
        self.boxsize = boxsize
        self.x = x
        self.m = m
        self.v = v

        if u is None:
            self.u = np.ones(self.N)
        else:
            self.u = u
        #print "Updating tree..."
        self.TreeUpdate()
        
        self.rho = self.des_ngb * self.m / (self.volnorm * self.h**self.dim)
        self.A = (gamma - 1)*self.u / self.rho**(gamma-1) 
        #print "Computing hydro force..."
        self.HydroForce()
    
    def Grad(self, f):
        df = DF(f, self.ngb)
        return np.einsum('ijk,ij->ik',self.dweights,df)
    
    def SymGrid(self, f):
        df = DF(f, self.ngb)
        return np.einsum('ijk,ij->ik',self.symweights,df)
    
    def TreeUpdate(self):
        self.tree = spatial.cKDTree(self.x, boxsize=self.boxsize)
        self.ngbdist, self.ngb = self.tree.query(self.x, self.des_ngb)
        #print "Computing smoothing length..."

        self.h = HsmlIter(self.ngbdist, error_norm=1e-13,dim=self.dim)

        #print "Computing grad weights..."
        t = time()
        self.q = np.einsum('i,ij->ij', 1/self.h, self.ngbdist)
    
        self.K = Kernel(self.q)
        self.dK = DKernel(self.q)
        self.weights = np.einsum('ij,i->ij',self.K, 1/np.sum(self.K,axis=1))
        
        self.dx = self.x[self.ngb] - self.x[:,None,:]
        Periodicize(self.dx.ravel(), self.boxsize)

        dx_matrix = np.einsum('ij,ijk,ijl->ikl', self.weights, self.dx,self.dx)
        dx_matrix = np.linalg.inv(dx_matrix)
        self.dweights = np.einsum('ikl,ijl,ij->ijk',dx_matrix, self.dx, self.weights)

        #self.divv = np.sum([self.Grad(v[:,i])[:,i] for i in xrange(self.des_ngb)])


        
    def HydroForce(self):
        self.rho = self.des_ngb * self.m / (self.volnorm * self.h**self.dim)
        self.P = self.A * self.rho**self.gamma
        self.F2 = np.einsum('ij,i->ij',-self.Grad(self.P), 1./self.rho)
        self.F = SymmetricForce(self.P, self.rho, self.ngb, self.dweights)
        
    def Timestep(self, dt):
        self.x = self.x + self.v * dt + self.F * dt**2 / 2
        self.x = self.x % self.boxsize % self.boxsize
        self.v = self.v + self.F * dt / 2
        #self.hsml_guess = self.h + dt * self.divv * self.h / self.dim
        self.TreeUpdate()
        self.HydroForce()
        self.v = self.v + self.F * dt / 2

In [44]:
res = (50,100,200,400)
print res
error = []
for N in res:
    print N
    x = np.array([np.arange(N,dtype=np.float64)/N]).T
    x = x + np.array([1e-6 * np.sin(2*np.pi*x[:,0]),]).T
    m = np.ones(N)/N
    v = np.zeros_like(x)
    v0 = v
    utherm = np.ones_like(m) * 0.4
    cs = (5./3 * 2./3 * utherm.mean())**0.5
    
    dtmin = 1. / 400 / cs * 0.2
    T = 1./cs
    nsteps = int(T/dtmin+1)
    dt = T/nsteps
    sys = ParticleSystem(m, x, v, utherm, des_ngb=4)
    rho0 = sys.rho
    for i in xrange(nsteps):
        sys.Timestep(dt)

    error.append(np.average(np.abs(sys.v - v0)))


(50, 100, 200, 400)
50
100
200
400


In [45]:
res, error

((50, 100, 200, 400),
 [2.6328256329756161e-08,
  6.6026938109275576e-09,
  1.649046271009425e-09,
  4.0925559871614743e-10])

In [50]:
from matplotlib import pyplot as plt
plt.plot(res,np.abs(error),marker='o')
plt.plot(res,np.min(error) * (np.float_(res)/max(res))**-2.0)
plt.loglog()
plt.show()

In [ ]:
@jit
def HexGrid(N):
    x = np.empty((N**2,2))
    for i in xrange(N):
        for j in xrange(N):
            x[i*N + j,0] = float(i)/N
            x[i*N + j,1] = float(j)/N + 1./(np.sqrt(3)*N)*(i%2)
    return x

In [87]:
import h5py
import numpy as np
from SmoothingLength import HsmlIter
from matplotlib import pyplot as plt
from GridDeposit import GridSurfaceDensityPeriodic
from scipy.spatial import cKDTree
F = h5py.File("gresho_ics.hdf5")
u = np.float64(F["PartType0"]["InternalEnergy"])
x = np.float64(F["PartType0"]["Coordinates"])[:,:2]
m = np.float64(F["PartType0"]["Masses"])
v = np.float64(F["PartType0"]["Velocities"])[:,:2]
#plt.scatter(np.sum((x-0.5)**2,axis=1)**0.5,np.sum(v**2,axis=1)**0.5)
#plt.show()
sys = ParticleSystem(m, x, v, u, gamma=1.4, des_ngb=20)
plt.scatter(np.sum((sys.x-0.5)**2,axis=1)**0.5, np.sum(sys.F**2,axis=1)**0.5)
#plt.scatter(np.sum((sys.x-0.5)**2,axis=1)**0.5, sys.P)

plt.show()
#plt.scatter(x[:,0],x[:,1])
#plt.xlim(0,1)
#plt.ylim(0,1)
#plt.show()
t = 0
i = 0
dtmin = (sys.h / np.sqrt(sys.u)).min() / 10
data = GridSurfaceDensityPeriodic(sys.m, sys.x-0.5, sys.h, 400, 1.0)
plt.imshow(data,cmap='RdBu',vmin=0.8,vmax=1.2)
plt.show()

print sys.x[(np.abs(np.array(F["PartType0"]["Density"]) - sys.rho)).argmax()]
while t < 0.0:  
    sys.Timestep(dtmin)

    t += dtmin
    i += 1
    if i%100==0: print t
    
plt.clf()
#plt.scatter(np.sum((sys.x-0.5)**2,axis=1)**0.5,np.sum(sys.F**2,axis=1)**0.5)
#plt.xlim(0,0.5)
#data = GridSurfaceDensityPeriodic(sys.m, sys.x, sys.h, 400, 1.0)
#plt.imshow(data)
#plt.show()

[ 0.00281957  0.58813596]


In [ ]:
plt.scatter(np.sum((sys.x-0.5)**2,axis=1)**0.5,np.sum(sys.v**2,axis=1)**0.5)
plt.xlim(0,0.5)
plt.show()

In [ ]:
from GridDeposit import Grid

In [ ]:
((5./3 * (5./3 - 1)) * 0.4)**0.5

In [ ]:
(2./3)**0.5